<a href="https://colab.research.google.com/github/chlin1/TimeSeries/blob/master/basic_time_series_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Forecasting
  - Data
  - Models
    - CNN
    - LSTM
    - CNN+LSTM

In [0]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import numpy as np
import pandas as pd

## Data

In [0]:
urlx = 'https://raw.githubusercontent.com/chlin1/TimeSeries/master/data/data_X.csv'
urly = 'https://raw.githubusercontent.com/chlin1/TimeSeries/master/data/data_Y.csv'
dfx = pd.read_csv(urlx)
dfy = pd.read_csv(urly)

# input parameters
sample = 1000 # number of samples
slide = 1 # stride 
subtimesteps = 25 # for CNN+LSTM
num_subtimesteps = 4 # for CNN+LSTM



timesteps = num_subtimesteps * subtimesteps # timesteps  


# reshape from [samples, timesteps] into [samples, timesteps, features]
datax=[]
datay=[]

for i in range(1,sample):
  datax.append(dfx.iloc[i*slide:i*slide + timesteps, 1:].values)
  datay.append(dfy.iloc[i*slide + timesteps - 1, 1:].values)
  
X = array(datax)
y = array(datay).flatten()

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
dummy_y = np_utils.to_categorical(encoded_Y)

## Models

In [0]:
def model_CNN(X,Y):
  
  num_sample, timesteps, num_feature = X.shape
  
  # define model
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timesteps, num_feature)))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(30, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit model
  model.fit(X, Y, epochs=1000, verbose=0)
  score, acc = model.evaluate(X,Y)

  print(acc)

In [0]:
def model_LSTM(X,Y):
  
  num_sample, timesteps, num_feature = X.shape
  
  # define model
  model = Sequential()
  model.add(LSTM(50, activation='relu', input_shape=(timesteps, num_feature)))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit model
  model.fit(X, Y, epochs=1000, verbose=0)
  score, acc = model.evaluate(X,Y)

  print(acc)

In [0]:
def mode_CNN_LSTM(X,Y):
  
  num_sample, timesteps, num_feature = X.shape
  
  # reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
  X = X.reshape((num_sample, num_subtimesteps , subtimesteps, num_feature))
  # define model
  model = Sequential()

  model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, subtimesteps, num_feature)))
  # input (sample, timesteps, features) output (sample, timesteps, fiters)
  # filters: the dimensionality of the output space (i.e. the number of output filters in the convolution)
  model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
  model.add(TimeDistributed(Flatten()))
  model.add(LSTM(50, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  # fit model
  model.fit(X, Y, epochs=500, verbose=0)
  score, acc = model.evaluate(X,Y)
  
  print(acc)

In [27]:
model_CNN(X,dummy_y)

999/999 [==============================] - 0s 335us/step
0.6216216216216216


In [28]:
model_LSTM(X,dummy_y)

999/999 [==============================] - 3s 3ms/step
0.24324324324324326


In [29]:
mode_CNN_LSTM(X,dummy_y)

999/999 [==============================] - 1s 625us/step
0.6196196196196196
